In [13]:
import pandas as pd
try:
    from sentence_transformers import SentenceTransformer
except:
    !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer
import faiss

In [14]:
df = pd.read_csv("../data/cleaned_job_descriptions.csv")

In [15]:
#confirm that the number of lines is equal to the number of rows in the dataframe
# df = df.drop_duplicates()
# df['Role'].unique()
# roles_to_keep = ['Frontend Web Developer', 'Wireless Network Engineer', 'User Interface Designer', 'Front-End Developer', 'IT Support Specialist', 'Database Developer', 'Social Science Researcher', 'Record Keeper',
#        'Staff Nurse', 'International Tax Consultant', 'Risk Analyst',
#        'Speech Pathologist', 'Regional Sales Director',
#        'Mental Health Counselor', 'Urban Planner',
#        'Strategic Account Manager', 'User Researcher',
#        'Financial Analyst', 'Desktop Support Technician' ]

# df = df[df['Role'].isin(roles_to_keep)]
# df['Role'].unique()
# df.shape

# df

#(1610462, 7)
#shape after filtering to keep only the roles we are interested in
#(86322, 7)
lines = []
for index, row in df.iterrows():
    row_string = " ".join(row.astype(str))  # Convert each value to string and join
    lines.append(row_string)

print(lines[0])


2 to 12 Years BCA Ashgabat Web Developer Frontend Web Developer Frontend Web Developers design and implement user interfaces for websites, ensuring they are visually appealing and user-friendly. They collaborate with designers and backend developers to create seamless web experiences for users. PNC Financial Services Group


In [16]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(lines, show_progress_bar=True)
embeddings.shape

Batches:   0%|          | 0/2698 [00:00<?, ?it/s]

(86327, 384)

In [ ]:
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
index.is_trained
print(type(index))


import joblib
#move the model and the index to the model folder
# print("type of model", type(model))
# print("type of index", type(index))

# #save using faiss
# faiss.write_index(index, "../model/faiss_index.faiss")
# #save using joblib
# joblib.dump(model, "../model/model.joblib")


<class 'faiss.swigfaiss.IndexFlatL2'>
type of model <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
type of index <class 'faiss.swigfaiss.IndexFlatL2'>


['../model/model.joblib']

In [23]:
k = 4
query = "I am a student looking for a job in the tech industry, I want to do cyber security"
query_embedding = model.encode(query)
query_embedding = query_embedding.reshape(1, -1)

In [24]:
D, I = index.search(query_embedding, k)
indices = I[0]
upper_bound = df.shape[0]

for index in indices:
    if index < upper_bound:
        print(df.iloc[index]['Job_Title'], '\n\n')
    else:
        print("No match found")

Systems Administrator 


UI Developer 


UI Developer 


Network Engineer 




In [ ]:
#save the model and index for later use using joblib or pickle
import joblib
print("type of model", type(model))
print("type of index", type(index))

#save using faiss
faiss.write_index(index, "../model/faiss_index.faiss")
#save using joblib
joblib.dump(model, "../model/model.joblib")





type of model <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
type of index <class 'numpy.int64'>


In [30]:
#save the dataframe as well
df.to_pickle("../model/df.pkl")